Município de São Paulo<br>
<span style="font-size:14px">Fonte: <a src="https://dados.gov.br/dados/conjuntos-dados/serie-historica-de-precos-de-combustiveis-e-de-glp">ANP</a><br></span>
1. Valor médio por ano de cada tipo de combustível
2. Valor médio por mês de cada tipo de combustível
3. Série temporal dos valores mínimos, média e máxima
4. Qual bandeira possui mais revendedoras?
5. Qual bandeira possui maior e menor preço médio?
6. Quantos litros é possível comprar com um salário mínimo?
7. Gastos com abastecimento (60Km diário, tanque de 50 litros, consumo 10km/litro)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import calendar
import locale

# <h1 style="font-size:16px">Importação e Transformação</h1>

In [ ]:
# Função para criar um calendário sem os finais de semana e feriados.
# Já que a análise é feita sobre a cidade de São Paulo, foi incluido o aniversário de São Paulo
def criar_calendario(ano_inicio:int, ano_fim:int):
    """
    Criar dataframe com calendário entre determinados anos. Não possui finais de
    semana ou feriados.

    Args:
        ano_inicio (int): Primeiro ano do calendário
        ano_fim (int): Último ano do calendário
    """
    def obter_nome_dia_semana(data):
        """
        Traduz o nome do dia da semana para pt_BR
        Args:
            data (str): Data
        Returns:
            str: Retorna o nome do dia da semana traduzido
        """

        # Especifica localização para obter o nome dos dias da semana em inglês
        # para posterior tradução
        locale.setlocale(locale.LC_TIME, "en_US.UTF-8")
        dias_semana_pt = {
            "Monday": "segunda-feira",
            "Tuesday": "terça-feira",
            "Wednesday": "quarta-feira",
            "Thursday": "quinta-feira",
            "Friday": "sexta-feira",
            "Saturday": "sábado",
            "Sunday": "domingo",
        }

        nome_dia_em_ingles = calendar.day_name[data.weekday()]

        return dias_semana_pt[nome_dia_em_ingles]

    def criar_calendario_diario_dataframe(ano_inicio, ano_fim):
        """
        Cria uma série de datas entre os anos especificados
        Args:
            ano_inicio (int): _description_
            ano_fim (int): _description_
        Returns:
            dataframe: Retorna um dataframe com as datas e dias da semana
        """

        # Cria um Pandas Series com as datas
        datas = pd.date_range(start=f"{ano_inicio}-01-01", end=f"{ano_fim}-12-31")

        # Transforma em um DataFrame
        calendario = pd.DataFrame(datas, columns=["Data"])

        # Adiciona coluna dia da semana com a função obter_nome_dia_semana
        calendario["Dia da Semana"] = calendario["Data"].apply(obter_nome_dia_semana)

        return calendario

    # Cria o dataframe
    calendario_df = criar_calendario_diario_dataframe(2022, 2024)
    
    # Remove os finais de semana
    calendario_df = calendario_df[
        ~calendario_df["Dia da Semana"].isin(["sábado", "domingo"])
    ]

    # Importa os feriados informados pela ANBIMA
    feriados = pd.read_excel(
        "https://www.anbima.com.br/feriados/arqs/feriados_nacionais.xls"
    )

    # Remove linhas com valores nulos, são textos que não são necessários
    feriados.dropna(inplace=True)

    # Transforma a coluna 'Data' para o tipo datetime
    feriados["Data"] = pd.to_datetime(feriados["Data"], format="%Y/%m/%d")
    
    # Filtra os feriados pelos anos do calendário
    feriados = feriados[
        (feriados["Data"].dt.year.isin(calendario_df["Data"].dt.year.unique()))
    ]

    # Inclui o aniversário de São Paulo em todos os anos
    for ano in calendario_df["Data"].dt.year.unique():
        aniv_sp = pd.to_datetime(f"{ano}-01-25")
        feriados.loc[len(feriados)] = [
            aniv_sp,
            obter_nome_dia_semana(aniv_sp).lower(),
            "Aniversário de São Paulo",
        ]

    # Ordena o calendário pelas datas
    feriados = feriados.sort_values(by="Data").reset_index(drop=True)

    # Remove os finais de semana
    feriados = feriados[~feriados["Dia da Semana"].isin(["sábado", "domingo"])]

    # Faz a união do calendário com os feriados
    calendario_df = (
        calendario_df.merge(feriados, on="Data", how="left")
        .drop(columns="Dia da Semana_y")
        .rename(columns={"Dia da Semana_x": "Dia da Semana"})
    )

    # Todos as datas que não forem feriados são considerados dias úteis
    calendario_df["Feriado"] = calendario_df["Feriado"].fillna("Dia Útil")
    
    # Filtra apenas pelos dias úteis
    calendario_df = calendario_df[calendario_df["Feriado"] == "Dia Útil"]
    
    # Remove a coluna Feriado, já possui apenas um valor
    calendario_df.drop(columns="Feriado", inplace=True)

    return calendario_df


In [ ]:
Fonte: "https://dados.gov.br/dados/conjuntos-dados/serie-historica-de-precos-de-combustiveis-e-de-glp"

# Import data
df_2022_1 = pd.read_csv("__datasets/combustiveis_2022_01.csv", sep=";", decimal=",")
df_2022_2 = pd.read_csv("__datasets/combustiveis_2022_02.csv", sep=";", decimal=",")
df_2023_1 = pd.read_csv("__datasets/combustiveis_2023_01.csv", sep=";", decimal=",")
df_2023_2 = pd.read_csv("__datasets/combustiveis_2023_02.csv", sep=";", decimal=",")
df_2024_1 = pd.read_csv("__datasets/combustiveis_2024_01.csv", sep=";", decimal=",")
df_2024_2 = pd.read_csv("__datasets/combustiveis_2024_02.csv", sep=";", decimal=",")

In [ ]:
# Filtra os dataframes pela cidade de São Paulo e remove as colunas de endereço
# que não terão utilidade nesta análise
def filtrar_df(df):
    """
    Filtra o dataframe pela cidade de São Paulo
    Args:
        df (dataframe): Dataframe

    Returns:
        dataframe: Retorna dataframe filtrado
    """
    df = df[df["Municipio"] == "SAO PAULO"].drop(
        columns=[
            "Regiao - Sigla",
            "Valor de Compra",
            "Unidade de Medida",
            "Nome da Rua",
            "Numero Rua",
            "Complemento",
            "Cep",
        ]
    )
    return df

df_2022_1 = filtrar_df(df_2022_1)
df_2022_2 = filtrar_df(df_2022_2)
df_2023_1 = filtrar_df(df_2023_1)
df_2023_2 = filtrar_df(df_2023_2)
df_2024_1 = filtrar_df(df_2024_1)
df_2024_2 = filtrar_df(df_2024_2)

In [ ]:
# Salário Mínimo no Brasil
salary_min = {"2022": 1212, "2023": 1320, "2024": 1412}

In [ ]:
# Faz a união de linhas dos dataframes
df = pd.concat(
    [df_2022_1, df_2022_2, df_2023_1, df_2023_2, df_2024_1, df_2024_2], 
    ignore_index=True
)

In [ ]:
# Renomeia algumas colunas para facilitar a análise
df.rename(
    columns={
        "Estado - Sigla": "Estado",
        "Data da Coleta": "Data"
    },
    inplace=True
)

In [ ]:
# Função para obter o número da semana dentro de um mês
def semana_do_mes(data):
    # Primeiro dia do mês
    primeiro_dia_do_mes = data.replace(day=1)

    # Calcula o dia da semana do primeiro dia do mês (0 = segunda-feira, 6 = domingo)
    dia_da_semana_primeiro_dia = primeiro_dia_do_mes.weekday()

    # Calcula o número da semana do mês
    numero_semana = (data.day + dia_da_semana_primeiro_dia - 1) // 7 + 1

    return numero_semana

# Converte Data para o formato datetime
df["Data"] = pd.to_datetime(df["Data"], format="%d/%m/%Y")

# Criar coluna do ano e mês
df["Ano"] = df["Data"].dt.year.astype(str)
df["Mes"] = df["Data"].dt.month

# Criar coluna do nome abreviado do mês
locale.setlocale(locale.LC_TIME, "pt_BR.UTF-8")
df["Mes Nome"] = df["Mes"].apply(lambda x: calendar.month_abbr[x])

# Cria coluna do número da semana de cada mês
df["Semana"] = df["Data"].apply(semana_do_mes)

# <h1 style="font-size:16px">Exportar para csv</h1>

In [ ]:
df.to_csv(
    "combustiveis_sp_sp.csv", header=True, index=False, sep=";", decimal=","
)

# <h1 style="font-size:16px">Informações Estatísticas</h1>

In [ ]:
# Informações estatísticas
for produto in df["Produto"].unique():
    print(produto.center(20, "="))
    for ano in df["Ano"].unique():
        df_desc = df[(df["Produto"] == produto) & (df["Ano"] == ano)]["Valor de Venda"].describe()
        df_desc.loc["IQR"] = df_desc.loc["75%"] - df_desc.loc["25%"]

        print(ano.center(20, " "))
        print(df_desc)
        print()

In [ ]:
# Presença de outliers
for produto in df["Produto"].unique():
    print(produto.center(30, "="))
    for ano in df["Ano"].unique():
        df_ = df[(df["Produto"] == produto) & (df["Ano"] == ano)]
        preco = df_["Valor de Venda"]
        qt_valores = len(preco)

        Q1 = preco.quantile(0.25)
        Q3 = preco.quantile(0.75)
        IQR = Q3 - Q1

        max_outliers = preco[(preco > Q3 + (IQR * 1.5))]
        porc_max = len(max_outliers) / qt_valores * 100
        min_outliers = preco[(preco < Q1 - (IQR * 1.5))]
        porc_min = len(min_outliers) / qt_valores * 100

        if len(max_outliers) >= 1:
            print(f"{ano}: {len(max_outliers)} ({porc_max:.1f}%) outliers acima!")

        else:
            print(f"{ano}: Não possui outliers acima")

        if len(min_outliers) >= 1:
            print(f"{ano}: {len(min_outliers)} ({porc_min:.1f}%) outliers abaixo!")

        else:
            print(f"{ano}: Não possui outliers abaixo")

        print()

# <h1 style="font-size:16px">Gráficos</h1>

In [ ]:
# Produtos únicos
produtos = df["Produto"].unique().tolist()

# Anos únicos
anos = df["Ano"].unique().tolist()

cores_anos = {
    anos[0]: "orange",
    anos[1]: "green",
    anos[2]: "red",
}

# Determinar cor para cada produto
cores_produto = {
    produtos[0]: "orange",
    produtos[1]: "green",
    produtos[2]: "brown",
    produtos[3]: "gold",
    produtos[4]: "black",
    produtos[5]: "lightcoral",
}

## <h1 style="font-size:16px">Scatter Plot</h1>

In [ ]:
fig = go.Figure()

for produto in produtos:
    df_ = df[df["Produto"] == produto]
    for ano in anos:
        filtrado = df_[df_["Ano"] == ano]
        fig.add_trace(
            go.Scatter(
                x=filtrado["Data"],
                y=filtrado["Valor de Venda"],
                name=f"{ano}",
                visible=(produto == produtos[0]),
                mode="markers",
                marker=dict(color=cores_anos[ano]),
                customdata=df_[["Revenda"]],
                hovertemplate=(  # Texto ao passar o mouse
                    "<b>Data:</b> %{x}<br>"
                    "<b>Valor:</b> R$ %{y:.2f}<br>"
                    "<b>Revenda:</b> %{customdata}<br>"
                ),
                showlegend=True,
            )
        )

updatemenu = [
    {
        "buttons": [
            {
                "label": produto,
                "method": "update",
                "args": [
                    {
                        "visible": [
                            i // len(anos) == list(produtos).index(produto)
                            for i in range(len(fig.data))
                        ]
                    }
                ],
            }
            for produto in produtos
        ],
        "direction": "down",
        "showactive": True,
        "x": 0.1,
        "xanchor": "left",
        "y": 1.1,
        "yanchor": "top",
    }
]

fig.update_layout(
    height=600,
    width=1000,
    title={
        "text": "Scatter Plot",  # Título do gráfico
        "y": 0.98,  # Posição vertical do título (0 a 1)
        "x": 0.5,  # Posição horizontal do título (0 a 1)
        "xanchor": "center",  # Ancoragem horizontal
        "yanchor": "top",  # Ancoragem vertical
        "font": {
            "family": "Arial, sans-serif",  # Família da fonte
            "size": 24,  # Tamanho da fonte
            "color": "black",  # Cor da fonte
            "weight": "bold",
        },
    },
    updatemenus=updatemenu,
    yaxis_title=None,
    xaxis_title=None,
    xaxis_showticklabels=True,
    plot_bgcolor="rgba(0,0,0,0)",
    margin={"l": 20, "r": 20, "t": 100, "b": 10},
)

# Configurar eixo Y
fig.update_yaxes(
    title_text="",
    tickfont=dict(family="Arial", size=11, color="black", weight="bold"),
    tickprefix="R$ ",
    tickformat=".2f",
)

fig.show()

## <h1 style="font-size:16px">BoxPlot</h1>

In [ ]:
fig = go.Figure()

for ano in anos:
    df_ = df[df["Ano"] == ano]
    for produto in produtos:
        filtrado = df_[df_["Produto"] == produto]
        fig.add_trace(
            go.Box(
                y=filtrado["Valor de Venda"],
                name=f"{produto}",
                visible=(ano==anos[0]),
                boxpoints="outliers",
                jitter=0.3, # Espalha os pontos para melhor visualização
                pointpos=-1.8 # Posição dos pontos em relação ao boxplot
            )
        )

updatemenu = [
    {
        "buttons": [
            {
                "method": "update",
                "label": ano,
                "args": [
                    {
                        "visible": [
                            a == ano
                            for a in anos
                            for _ in produtos
                        ]
                    }
                ],
            }
            for ano in anos
        ],
        "direction": "down",
        "showactive": True,
        "x": 0.1,
        "xanchor": "left",
        "y": 1.15,
        "yanchor": "top",
    }
]

fig.update_layout(
    height=600,
    width=1000,
    title={
        "text": "Box Plot",  # Título do gráfico
        "y": 0.98,  # Posição vertical do título (0 a 1)
        "x": 0.5,  # Posição horizontal do título (0 a 1)
        "xanchor": "center",  # Ancoragem horizontal
        "yanchor": "top",  # Ancoragem vertical
        "font": {
            "family": "Arial, sans-serif",  # Família da fonte
            "size": 24,  # Tamanho da fonte
            "color": "black",  # Cor da fonte
            "weight": "bold",
        },
    },
    updatemenus=updatemenu,
    yaxis_title=None,
    xaxis_title=None,
    xaxis_showticklabels=False,
    plot_bgcolor="rgba(0,0,0,0)",
    margin={"l": 20, "r": 20, "t": 100, "b": 10},
)

# Configurar eixo Y
fig.update_yaxes(
    title_text="",
    tickfont=dict(family="Arial", size=11, color="black", weight="bold"),
    tickprefix="R$ ",
    tickformat=".2f",
)

fig.show()

In [ ]:
# Retirar os outliers identificados

outlier_diesel_s10 = df[df["Produto"] == "DIESEL S10"]["Valor de Venda"].max().tolist()

outlier_diesel = df[df["Produto"] == "DIESEL S10"]["Valor de Venda"].min().tolist()

df = df[
    (df["Valor de Venda"] != outlier_diesel_s10)
    & (df["Valor de Venda"] != outlier_diesel_s10)
]

## <h1 style="font-size:16px">Gráfico Densidade</h1>

In [ ]:
# Plota um gráfico de densidade para cada produto e ano
from matplotlib.ticker import MultipleLocator

fig, axs = plt.subplots(len(produtos), len(anos), figsize=(15, 15))

for a, produto in enumerate(produtos):
    df_ = df[df["Produto"] == produto]
    plt.title(produto)

    for b, ano in enumerate(anos):

        df_2 = df_[df_["Ano"] == ano]["Valor de Venda"]
        sns.histplot(
            x=df_2,
            ax=axs[a, b],
            label=ano,
            kde=True,
            stat="density",
            binwidth=0.5
        )

        # Título de cada gráfico
        axs[a, b].set_title(ano, weight="bold")
        
        # Intervalo entre valores do eixo X
        axs[a,b].xaxis.set_major_locator(MultipleLocator(0.5))

    # Título de cada linha
    axs[a, 0].set_ylabel(produtos[a], size="medium", weight="bold")

# Título principal
plt.suptitle("Gráfico de Densidade", size=20, weight="bold", y=1)

# Ajustar margens
plt.tight_layout()
plt.show()

## <h1 style="font-size:16px">Série Temporal da Média Móvel do Preço Médio</h1>

In [ ]:
# Agrupar por Produto e Data e retornar valores máximo, média e mínimo
df_grouped = (
    df.groupby(["Produto", "Data"])["Valor de Venda"]
    .agg(["max", "mean", "min"])
    .reset_index()
)

# Iniciar instância da figura
fig = go.Figure()

produto_inicial = "GASOLINA"
mov_avg_window = 7

def plot_line(df, y:str, name:str, visible):
    plot = go.Scatter(
        x=df["Data"],
        y=df[y].rolling(window=mov_avg_window).mean(),
        name=name, # Nome de cada linha
        mode="lines",
        visible=visible, # Visibilidade do gráfico
        customdata=df[["Produto"]], # Informações adicionais
        hovertemplate=(  # Texto ao passar o mouse
            "<b>Medida:</b> %{fullData.name}<br>"
            "<b>Data:</b> %{x}<br>"
            "<b>Valor:</b> R$ %{y:.2f}<br>"
            "<b>Produto:</b> %{customdata}<extra></extra>"
        ),
    )
    return plot

# Para cada produto irá gerar um gráfico da Data pelo Valor de Venda
for produto in produtos:
    filtrado = df_grouped[df_grouped["Produto"] == produto]

    # Define o produto que irá aparecer inicialmente
    visible = produto == produto_inicial

    # Plota o máximo dos valores
    trace1 = plot_line(filtrado, "max", "Máxima", visible)

    # Plota a média dos valores
    trace2 = plot_line(filtrado, "mean", "Média", visible)
    
    # Plota o mínimo dos valores
    trace3 = plot_line(filtrado, "min", "Mínima", visible)
    
    # Plota todas as linhas em um mesmo gráfico
    fig.add_traces([trace1, trace2, trace3])

# Configurar botões dos filtros
buttons = [
    dict(
        label=f"{produto}",
        method="update",
        args=[
            {"visible": [(produto == cat) for cat in produtos for _ in range(3)]},
        ],
    )
    for produto in produtos
]

# Configurações do gráfico
fig.update_layout(
    height=600,
    width=1200,
    title={
        "text": f"Média Móvel {mov_avg_window}-0",  # Título do gráfico
        "y": 0.98,  # Posição vertical do título (0 a 1)
        "x": 0.5,  # Posição horizontal do título (0 a 1)
        "xanchor": "center",  # Ancoragem horizontal
        "yanchor": "top",  # Ancoragem vertical
        "font": {
            "family": "Arial, sans-serif",
            "size": 26,
            "color": "black",
            "weight": "bold",
        },
    },
    margin=dict(l=50, r=20, t=100, b=0),  # Margem da área de plotagem
    updatemenus=[  # Botões do Filtro. Método menos eficiente
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
            direction="left",
            x=0.5,  # Posição horizontal dos botões (0 a 1)
            y=1.1,  # Posição vertical dos botões (0 a 1)
            xanchor="center",
            yanchor="top",
            active=1,
        )
    ],
)

# Configurar eixo X
fig.update_xaxes(
    title_text="",
    tickfont=dict(family="Arial", size=12, color="black", weight="bold"),
    range=[
        df_grouped["Data"].min() - timedelta(days=25),
        df_grouped["Data"].max() + timedelta(days=12),
    ],
)

# Configurar eixo Y
fig.update_yaxes(
    title_text="",
    tickfont=dict(family="Arial", size=11, color="black", weight="bold"),
    tickprefix="R$ ",
    tickformat=".2f",
)

fig.show()

## <h1 style="font-size:16px">Preço Médio Por Mês</h1>

In [ ]:
# Agrupar dados pelo produto, ano, mês (apenas para ajudar na ordenação) e nome do mês
df_grouped = (
    df.groupby(["Produto", "Ano", "Mes", "Mes Nome"])
    .agg({"Valor de Venda": "mean"})
    .reset_index()
)

# Arrendoda os valores
df_grouped["Valor de Venda"] = df_grouped["Valor de Venda"].apply(lambda x: round(x, 2))

# Concatena o nome do mês com o ano
df_grouped["Mes/Ano"] = (
    df_grouped["Mes Nome"] + "/" + [x[2:] for x in df_grouped["Ano"]]
)

# Iniciar instância da figura
fig = go.Figure()

# produto inicial
produto_inicial = "GASOLINA"

# Cria listas com valores mínimos e máximos para inserir no gráfico como anotação
max_valor = {}
min_valor = {}

# Para cada produto irá gerar um gráfico da Data pelo Valor de Venda
for produto in produtos:
    filtrado = df_grouped[df_grouped["Produto"] == produto]

    # Definir visibilidade inicial
    visible = produto == produto_inicial
    
    # Gerar valor máximo
    dfmax = filtrado[
        filtrado["Valor de Venda"] == filtrado["Valor de Venda"].max()
    ]
    
    # Gerar valor mínimo
    dfmin = filtrado[
        filtrado["Valor de Venda"] == filtrado["Valor de Venda"].min()
    ]

    # Adicionar a lista para cada produto (iteração do for)
    max_valor.update(
        {
            produto:{
                "Mes/Ano": dfmax["Mes/Ano"].tolist()[0],
                "Valor de Venda": dfmax["Valor de Venda"].tolist()[0]
            }
        }
    )
    min_valor.update(
        {
            produto:{
                "Mes/Ano": dfmin["Mes/Ano"].tolist()[0],
                "Valor de Venda": dfmin["Valor de Venda"].tolist()[0]
            }
        }
    )
    
    # Adicionar linha ao gráfico
    fig.add_trace(
        go.Scatter(
            x=filtrado["Mes/Ano"],
            y=filtrado["Valor de Venda"],
            name=produto,
            mode="lines",
            visible=visible, # Visibilidade inicial
            customdata=filtrado[["Produto"]], # Informações adicionais
            hovertemplate=( # Texto ao passar o mouse
                "<b>Medida:</b> %{fullData.name}<br>"
                "<b>Data:</b> %{x}<br>"
                "<b>Valor:</b> R$ %{y:.2f}<br>"
                "<b>Produto:</b> %{customdata}<extra></extra>"
            ),
        )
    )

# Função para criar anotação
def annotation(valor_dict, produto):

    dict_valor = valor_dict[produto]

    annotations = dict(
        x=dict_valor["Mes/Ano"],
        y=dict_valor["Valor de Venda"],
        text=f"R$ {dict_valor["Valor de Venda"]}",
    )

    return annotations

# Adcionar anotações que serão mostradas ao plotar o gráfico
fig.add_annotation(annotation(max_valor, produto_inicial))
fig.add_annotation(annotation(min_valor, produto_inicial))

# Parâmetros dos botões de filtro e anotações
# Toda vez que selecionar um botão irá alterar a linha plotada e a anotação
buttons = [
    dict(
        label=produtos[0],
        method="update",
        args=[
            {"visible": [True, False, False, False, False, False]},
            {
                "annotations": [
                    annotation(max_valor, produtos[0]),
                    annotation(min_valor, produtos[0]),
                ]
            },
        ],
    ),
    dict(
        label=produtos[1],
        method="update",
        args=[
            {"visible": [False, True, False, False, False, False]},
            {
                "annotations": [
                    annotation(max_valor, produtos[1]),
                    annotation(min_valor, produtos[1]),
                ]
            },
        ],
    ),
    dict(
        label=produtos[2],
        method="update",
        args=[
            {"visible": [False, False, True, False, False, False]},
            {
                "annotations": [
                    annotation(max_valor, produtos[2]),
                    annotation(min_valor, produtos[2]),
                ]
            },
        ],
    ),
    dict(
        label=produtos[3],
        method="update",
        args=[
            {"visible": [False, False, False, True, False, False]},
            {
                "annotations": [
                    annotation(max_valor, produtos[3]),
                    annotation(min_valor, produtos[3]),
                ]
            },
        ],
    ),
    dict(
        label=produtos[4],
        method="update",
        args=[
            {"visible": [False, False, False, False, True, False]},
            {
                "annotations": [
                    annotation(max_valor, produtos[4]),
                    annotation(min_valor, produtos[4]),
                ]
            },
        ],
    ),
    dict(
        label=produtos[5],
        method="update",
        args=[
            {"visible": [False, False, False, False, False, True]},
            {
                "annotations": [
                    annotation(max_valor, produtos[5]),
                    annotation(min_valor, produtos[5]),
                ]
            },
        ],
    ),
]


# Configurações do gráfico
fig.update_layout(
    height=500, # altura do gráfico
    width=1500, # largura do gráfico
    title={
        "text": f"Preço Médio Mensal",  # Título do gráfico
        "y": 0.97,  # Posição vertical do título (0 a 1)
        "x": 0.5,  # Posição horizontal do título (0 a 1)
        "xanchor": "center",  # Ancoragem horizontal
        "yanchor": "top",  # Ancoragem vertical
        "font": {
            "family": "Arial, sans-serif",
            "size": 26,
            "color": "black",
            "weight": "bold",
        },
    },
    margin=dict(l=20, r=20, t=100, b=20),  # Margem da área de plotagem
    updatemenus=[
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
            direction="left",
            x=0.5,  # Posição horizontal dos botões (0 a 1)
            y=1.15,  # Posição vertical dos botões (0 a 1)
            xanchor="center",
            yanchor="top",
            active=1,
        )
    ],
    yaxis=dict(tickprefix="R$ ", tickformat=".2f", title=""),
    xaxis=dict(tickfont=dict(size=11, color="black"), tickangle=45),
)

fig.show()

## <h1 style="font-size:16px">Preço Médio Por Ano</h1>

In [ ]:
# Agrupar por produto e ano
df_ = (
    df
    .groupby(["Produto", "Ano"])
    .agg({"Valor de Venda": "mean"})
    .reset_index()
)

# Iniciar instância da figura
fig = go.Figure()

produto_inicial = "GASOLINA"


# Para cada produto irá gerar um gráfico da Data pelo Valor de Venda
for produto in produtos:
    filtrado = df_[df_["Produto"] == produto]
    visible = produto == produto_inicial
    fig.add_trace(
        go.Bar(
            y=filtrado["Ano"],
            x=filtrado["Valor de Venda"],
            name=produto,
            orientation="h",
            text=filtrado["Valor de Venda"],
            textposition="outside",
            texttemplate="R$ %{x:.2f}",
            textfont=dict(family="Arial", size=12, color="black"),
            visible=visible,
            customdata=filtrado[["Produto"]],
            hovertemplate=(
                "<b>Ano:</b> %{y}<br>"
                "<b>Valor:</b> R$ %{x:.2f}<br>"
                "<b>Produto:</b> %{customdata}<extra></extra>"
            ),
        )
    )

def xlim(produto):
    xlim = df_[df_["Produto"] == produto]["Valor de Venda"].max().tolist()
    lim = [-0.1, xlim * 1.08]
    return lim

buttons = [
    dict(
        label=produtos[0],
        method="update",
        args=[
            {"visible": [True, False, False, False, False, False]},
            {"xaxis.range": xlim(produtos[0])},
        ],
    ),
    dict(
        label=produtos[1],
        method="update",
        args=[
            {"visible": [False, True, False, False, False, False]},
            {"xaxis.range": xlim(produtos[1])},
        ],
    ),
    dict(
        label=produtos[2],
        method="update",
        args=[
            {"visible": [False, False, True, False, False, False]},
            {"xaxis.range": xlim(produtos[2])},
        ],
    ),
    dict(
        label=produtos[3],
        method="update",
        args=[
            {"visible": [False, False, False, True, False, False]},
            {"xaxis.range": xlim(produtos[3])},
        ],
    ),
    dict(
        label=produtos[4],
        method="update",
        args=[
            {"visible": [False, False, False, False, True, False]},
            {"xaxis.range": xlim(produtos[4])},
        ],
    ),
    dict(
        label=produtos[5],
        method="update",
        args=[
            {"visible": [False, False, False, False, False, True]},
            {"xaxis.range": xlim(produtos[5])},
        ],
    ),
]

# Configurações do gráfico
fig.update_layout(
    height=400,
    width=1000,
    title={
        "text": f"Preço Médio Anual",  # Título do gráfico
        "y": 0.97,  # Posição vertical do título (0 a 1)
        "x": 0.5,  # Posição horizontal do título (0 a 1)
        "xanchor": "center",  # Ancoragem horizontal
        "yanchor": "top",  # Ancoragem vertical
        "font": {
            "family": "Arial, sans-serif",
            "size": 26,
            "color": "black",
            "weight": "bold",
        },
    },
    margin=dict(l=50, r=20, t=100, b=20),  # Margem da área de plotagem
    updatemenus=[  # Botões do Filtro. Método menos eficiente
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
            direction="left",
            x=0.5,  # Posição horizontal dos botões (0 a 1)
            y=1.2,  # Posição vertical dos botões (0 a 1)
            xanchor="center",
            yanchor="top",
            active=1,
        )
    ],
    xaxis=dict(showticklabels=False, title="", range=xlim(produto_inicial)),
    yaxis=dict(
        tickfont=dict(
            family="Arial",  # Escolha da fonte
            size=14,  # Tamanho da fonte
            color="black",  # Cor da fonte
            weight="bold",  # Aplicação de negrito (não diretamente suportado)
        )
    ),
)

fig.show()

## <h1 style="font-size:16px">Bandeira com mais revendedores</h1>

In [ ]:
# Agrupar por Bandeira e produto
df_ = df.groupby(["Bandeira", "Produto"]).agg({"Revenda": "nunique"}).reset_index()

fig = go.Figure()

produto_inicial = "GASOLINA"

for produto in produtos:
    filtrado = df_[df_["Produto"] == produto]

    # Obter o total de revendedores
    total_revenda = filtrado["Revenda"].sum()

    # Ordenar pela quantidade de revendedores, ascendente
    filtrado = filtrado.sort_values(by="Revenda")

    # Criar coluna com a porcentagem do total dos revendedores
    filtrado["Porcent"] = filtrado["Revenda"].div(total_revenda).mul(100)
    visible = produto == produto_inicial
    fig.add_trace(
        go.Bar(
            x=filtrado["Porcent"],
            y=filtrado["Bandeira"],
            orientation="h",
            name=produto,
            text=filtrado["Porcent"],
            textposition="outside",
            texttemplate="%{x:.2f}%",
            textfont=dict(family="Arial", size=12, color="black"),
            visible=visible,
            hoverinfo="none",
        )
    )


def xlim(produto):
    df1 = df_[df_["Produto"] == produto]
    df1_sum = df1["Revenda"].sum()
    xlim = df1["Revenda"].div(df1_sum).mul(100).max().tolist()
    lim = [-0.5, xlim * 1.1]
    return lim


buttons = [
    dict(
        label=produtos[0],
        method="update",
        args=[
            {"visible": [True, False, False, False, False, False]},
            {"xaxis.range": xlim(produtos[0])},
        ],
    ),
    dict(
        label=produtos[1],
        method="update",
        args=[
            {"visible": [False, True, False, False, False, False]},
            {"xaxis.range": xlim(produtos[1])},
        ],
    ),
    dict(
        label=produtos[2],
        method="update",
        args=[
            {"visible": [False, False, True, False, False, False]},
            {"xaxis.range": xlim(produtos[2])},
        ],
    ),
    dict(
        label=produtos[3],
        method="update",
        args=[
            {"visible": [False, False, False, True, False, False]},
            {"xaxis.range": xlim(produtos[3])},
        ],
    ),
    dict(
        label=produtos[4],
        method="update",
        args=[
            {"visible": [False, False, False, False, True, False]},
            {"xaxis.range": xlim(produtos[4])},
        ],
    ),
    dict(
        label=produtos[5],
        method="update",
        args=[
            {"visible": [False, False, False, False, False, True]},
            {"xaxis.range": xlim(produtos[5])},
        ],
    ),
]

fig.update_layout(
    height=600,
    width=900,
    title={
        "text": "Qnt Revendedoras",  # Título do gráfico
        "y": 0.98,  # Posição vertical do título (0 a 1)
        "x": 0.5,  # Posição horizontal do título (0 a 1)
        "xanchor": "center",  # Ancoragem horizontal
        "yanchor": "top",  # Ancoragem vertical
        "font": {
            "family": "Arial, sans-serif",  # Família da fonte
            "size": 24,  # Tamanho da fonte
            "color": "black",  # Cor da fonte
            "weight": "bold",
        },
    },
    updatemenus=[  # Botões do Filtro. Método menos eficiente
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
            direction="left",
            x=0.5,  # Posição horizontal dos botões (0 a 1)
            y=1.1,  # Posição vertical dos botões (0 a 1)
            xanchor="center",
            yanchor="top",
            active=1,
        )
    ],
    yaxis_title=None,
    xaxis=dict(showticklabels=False, title="", range=xlim(produto_inicial)),
    plot_bgcolor="rgba(0,0,0,0)",
    margin={"l": 20, "r": 20, "t": 100, "b": 10},
)

fig.show()

## <h1 style="font-size:16px">Preço Médio por Bandeira em 2024</h1>

In [ ]:
# Agrupar por Bandeira e Produto e obter valores apenas de 2024
df_ = (
    df[df["Ano"] == "2024"]
    .groupby(["Bandeira", "Produto"])
    .agg({"Valor de Venda": "mean"})
    .reset_index()
)

fig = go.Figure()

produto_inicial = "GASOLINA"

for produto in produtos:
    filtrado = df_[df_["Produto"] == produto]
    filtrado = filtrado.sort_values(by="Valor de Venda")
    visible = produto == produto_inicial
    fig.add_trace(
        go.Bar(
            x=filtrado["Valor de Venda"],
            y=filtrado["Bandeira"],
            orientation="h",
            name=produto,
            text=filtrado["Valor de Venda"],
            textposition="outside",
            texttemplate="R$ %{x:.2f}",
            textfont=dict(family="Arial", size=12, color="black"),
            visible=visible,
            hoverinfo="none"
        )
    )


def xlim(produto):
    xlim = df_[df_["Produto"] == produto]["Valor de Venda"].max()
    lim = [-0.1, xlim * 1.1]
    return lim


buttons = [
    dict(
        label=produtos[0],
        method="update",
        args=[
            {"visible": [True, False, False, False, False, False]},
            {"xaxis.range": xlim(produtos[0])},
        ],
    ),
    dict(
        label=produtos[1],
        method="update",
        args=[
            {"visible": [False, True, False, False, False, False]},
            {"xaxis.range": xlim(produtos[1])},
        ],
    ),
    dict(
        label=produtos[2],
        method="update",
        args=[
            {"visible": [False, False, True, False, False, False]},
            {"xaxis.range": xlim(produtos[2])},
        ],
    ),
    dict(
        label=produtos[3],
        method="update",
        args=[
            {"visible": [False, False, False, True, False, False]},
            {"xaxis.range": xlim(produtos[3])},
        ],
    ),
    dict(
        label=produtos[4],
        method="update",
        args=[
            {"visible": [False, False, False, False, True, False]},
            {"xaxis.range": xlim(produtos[4])},
        ],
    ),
    dict(
        label=produtos[5],
        method="update",
        args=[
            {"visible": [False, False, False, False, False, True]},
            {"xaxis.range": xlim(produtos[5])},
        ],
    ),
]

fig.update_layout(
    height=500,
    width=900,
    title={
        "text": "Preço Médio por Bandeira - 2024",  # Título do gráfico
        "y": 0.98,  # Posição vertical do título (0 a 1)
        "x": 0.5,  # Posição horizontal do título (0 a 1)
        "xanchor": "center",  # Ancoragem horizontal
        "yanchor": "top",  # Ancoragem vertical
        "font": {
            "family": "Arial, sans-serif",  # Família da fonte
            "size": 24,  # Tamanho da fonte
            "color": "black",  # Cor da fonte
            "weight": "bold",
        },
    },
    updatemenus=[  # Botões do Filtro. Método menos eficiente
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
            direction="left",
            x=0.5,  # Posição horizontal dos botões (0 a 1)
            y=1.14,  # Posição vertical dos botões (0 a 1)
            xanchor="center",
            yanchor="top",
            active=1,
        )
    ],
    yaxis_title=None,
    xaxis=dict(showticklabels=False, title="", range=xlim(produto_inicial)),
    xaxis_showticklabels=False,
    plot_bgcolor="rgba(0,0,0,0)",
    margin={"l": 20, "r": 20, "t": 100, "b": 10},
)

fig.show()

# <h1 style="font-size:16px">Gasto com Abastecimento cada X dias</h1>

## <h1 style="font-size:16px">Funções</h1>

In [ ]:
def abastecimentos(df_preco, df_calendario, produto, tanque, km_litro, distancia_dia):
    """Cria um dataframe que calcula o valor para abastecer a cada x dias

    Args:
        df_preco (_type_): _description_
        df_calendario (_type_): _description_
        produto (_type_): _description_
        intervalo (_type_): _description_

    Raises:
        TypeError: _description_

    Returns:
        _type_: _description_
    """
    df_produto = df_preco[df_preco["Produto"] == produto]
    df_abast = df_calendario.merge(df_produto, on="Data", how="left").bfill()
    df_abast.loc[0, "Data"] = df_abast.loc[0, "Data"] + timedelta(hours=9)
    df_abast.set_index("Data", inplace=True)
    
    def intervalo_dias(produto, tanque, km_litro, distancia_dia):
        intervalo = round((tanque * km_litro[produto] / distancia_dia), 1)
        return pd.Timedelta(days=intervalo)
    
    intervalo = intervalo_dias(produto, tanque, km_litro, distancia_dia)

    # Função para ajustar a data e hora
    def ajustar_data_hora(data):
        """
        Ajusta a data e hora:
        - Se a hora for antes das 07:00, define como 07:00.
        - Se a hora for depois das 18:00, avança para o próximo dia útil e soma a diferença às 07:00.

        Args:
            data (Timestamp): Data no formato Timestamp

        Raises:
            TypeError: Se data não for do formato Timestamp retorna TypeError
        Returns:
            Timestamp: Data Hora ajustado
        """
        # Define os horários de referência
        hora_inicio = pd.Timestamp("07:00").time()
        hora_fim = pd.Timestamp("18:00").time()

        # Extrai a hora da data
        if isinstance(data, pd.Timestamp):  # Verifica se é um objeto Timestamp
            hora = data.time()
        else:
            raise TypeError("O objeto 'data' deve ser do tipo pd.Timestamp.")

        if hora < hora_inicio:  # Se antes das 07:00
            data = data.replace(hour=7, minute=0, second=0, microsecond=0)
        elif hora > hora_fim:  # Se depois das 18:00
            # Calcula a diferença entre o horário e 18:00
            diferenca = data - data.replace(hour=18, minute=0, second=0, microsecond=0)
            # Avança para o próximo dia útil
            data = data + pd.Timedelta(days=1)
            # Define como 07:00
            data = data.replace(
                hour=7, minute=0, second=0, microsecond=0
            )
            # Soma a diferença às 07:00
            data += diferenca
            
            # Verifica se o novo dia é um final de semana e avança para o próximo dia útil
            while data.weekday() >= 5:  # Sábado (5) ou Domingo (6)
                data += pd.Timedelta(days=1)
        return data

    # Gerar uma nova série de datas com os intervalos
    datas_selecionadas = []
    data_atual = df_abast.index.min()  # Começa na primeira data do DataFrame

    while data_atual <= df_abast.index.max():
        # Ajusta a data e hora
        data_atual = ajustar_data_hora(data_atual)
        datas_selecionadas.append(data_atual)
        data_atual += intervalo # Incrementa a data atual com o intervalo

    # Selecionar os valores correspondentes no DataFrame
    abastecimento = df_abast.reindex(datas_selecionadas, method="nearest")

    abastecimento["Total"] = abastecimento["Valor de Venda"].mul(tanque)

    abastecimento.reset_index(inplace=True)
    
    

    return abastecimento

## <h1 style="font-size:16px">Variáveis Iniciais</h1>

In [ ]:
tanque = 50
km_litro = {
    "GASOLINA": 12,
    "ETANOL": 9,
    "DIESEL": 15,
    "DIESEL S10": 16,
    "GASOLINA ADITIVADA": 12,
    "GNV": 15,
}
dist_dia = 60

calendario_df = criar_calendario(2022, 2024)
df_ = df.groupby(["Data", "Produto"]).agg({"Valor de Venda": "mean"}).reset_index()

## <h1 style="font-size:16px">Série Temporal</h1>

In [ ]:
# Iniciar instância da figura
fig = go.Figure()

# Produto que aparece ao plotar o gráfico
produto_inicial = "GASOLINA"


# Para cada produto irá gerar um gráfico da Data pelo Valor de Venda
for produto in produtos:
    
    filtrado = abastecimentos(
        df_, calendario_df, produto, tanque, km_litro, dist_dia)

    visible = produto == produto_inicial

    fig.add_trace(

        go.Scatter(

            x=filtrado["Data"],

            y=filtrado["Total"],
            name=produto,

            visible=visible,
            mode="lines",

            hovertemplate=("<b>Data:</b> %{x}<br>" "<b>Valor:</b> R$ %{y:.2f}<br>"),

        )

    )

def anotacoes_consumo(produto):
    anotacao = dict(
        text=(
            f"Capacidade Tanque: {tanque} Litros<br>"
            f"Distância por Dia: {dist_dia} Km<br>"
            f"Consumo Veículo: {km_litro[produto]} Km/l<br>"
            f"Abastecimento a cada: {round(tanque * km_litro[produto] / dist_dia, 1):.1f} dias"
        ),
        xref="paper",
        yref="paper",
        x=0,  # posição x da caixa de texto
        y=1.18,  # posição y da caixa de texto
        showarrow=False,
    )
    return anotacao


# Adicione uma anotação (caixa de texto)
fig.add_annotation(
    anotacoes_consumo(produto_inicial)
)


buttons = [
    dict(
        label=produtos[0],
        method="update",
        args=[
            {"visible": [True, False, False, False, False, False]},
            {"annotations": [anotacoes_consumo(fig.data[0].name)]}
        ],
    ),
    dict(
        label=produtos[1],
        method="update",
        args=[
            {"visible": [False, True, False, False, False, False]},
            {"annotations": [anotacoes_consumo(fig.data[1].name)]}
        ],
    ),
    dict(
        label=produtos[2],
        method="update",
        args=[
            {"visible": [False, False, True, False, False, False]},
            {"annotations": [anotacoes_consumo(fig.data[2].name)]}
        ],
    ),
    dict(
        label=produtos[3],
        method="update",
        args=[
            {"visible": [False, False, False, True, False, False]},
            {"annotations": [anotacoes_consumo(fig.data[3].name)]}
        ],
    ),
    dict(
        label=produtos[4],
        method="update",
        args=[
            {"visible": [False, False, False, False, True, False]},
            {"annotations": [anotacoes_consumo(fig.data[4].name)]}
        ],
    ),
    dict(
        label=produtos[5],
        method="update",
        args=[
            {"visible": [False, False, False, False, False, True]},
            {"annotations": [anotacoes_consumo(fig.data[5].name)]}
        ],
    ),
]

# Configurações do gráfico
fig.update_layout(
    height=600,
    width=1300,
    title={
        "text": "Gasto Médio com Abastecimento",  # Título do gráfico
        "y": 0.98,  # Posição vertical do título (0 a 1)
        "x": 0.5,  # Posição horizontal do título (0 a 1)
        "xanchor": "center",  # Ancoragem horizontal
        "yanchor": "top",  # Ancoragem vertical
        "font": {
            "family": "Arial, sans-serif",
            "size": 26,
            "color": "black",
            "weight": "bold",
        },
    },
    margin=dict(l=50, r=20, t=100, b=0),  # Margem da área de plotagem
    updatemenus=[  # Botões do Filtro. Método menos eficiente
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
            direction="left",
            x=0.5,  # Posição horizontal dos botões (0 a 1)
            y=1.1,  # Posição vertical dos botões (0 a 1)
            xanchor="center",
            yanchor="top",
            active=1,
        )
    ],
)

# Configurar eixo X
fig.update_xaxes(
    title_text="",
    tickfont=dict(family="Arial", size=12, color="black", weight="bold"),
    range=[
        df_["Data"].min() - timedelta(days=25),
        df_["Data"].max() + timedelta(days=12),
    ],
)

# Configurar eixo Y
fig.update_yaxes(
    title_text="",
    tickfont=dict(family="Arial", size=12, color="black", weight="bold"),
    tickprefix="R$ ",
    tickformat=".0f"
)

fig.show()

## <h1 style="font-size:16px">Série Temporal Mensal</h1>

In [ ]:
# Iniciar instância da figura
fig = go.Figure()

# Produto que aparece ao plotar o gráfico
produto_inicial = "GASOLINA"


# Para cada produto irá gerar um gráfico da Data pelo Valor de Venda
for produto in produtos:

    filtrado = abastecimentos(df_, calendario_df, produto, tanque, km_litro, dist_dia)
    filtrado["Ano"] = filtrado["Data"].dt.year
    filtrado["Mes"] = filtrado["Data"].dt.month
    filtrado["Mes Nome"] = filtrado["Mes"].apply(lambda x: calendar.month_abbr[x])
    filtrado["Mes/Ano"] = filtrado["Mes Nome"] + "/" + filtrado["Ano"].astype(str)
    filtrado["Qnt"] = 1
    filtrado = (
        filtrado
        .groupby(["Ano", "Mes", "Mes/Ano"])
        .agg({"Total":"sum", "Qnt":"sum"})
        .reset_index()
    )
    visible = produto == produto_inicial
    fig.add_trace(
        go.Scatter(
            x=filtrado["Mes/Ano"],
            y=filtrado["Total"],
            name=produto,
            visible=visible,
            customdata=filtrado[["Qnt"]],
            mode="lines",
            hovertemplate=(
                "<b>Data:</b> %{x}<br>"
                "<b>Valor:</b> R$ %{y:.2f}<br>"
                "<b>Qnt Abastecimento:</b> %{customdata}"
            ),
        )
    )

# Adicione uma anotação (caixa de texto)
fig.add_annotation(
    text=(
        f"Capacidade Tanque: {tanque} Litros<br>"
        f"Distância por Dia: {dist_dia} Km<br>"
        f"Consumo Veículo: {km_litro[produto_inicial]} Km/l<br>"
        f"Abastecimento a cada: {round((tanque * km_litro[produto_inicial] / dist_dia), 1)} dias"
    ),
    xref="paper",
    yref="paper",
    x=0,  # posição x da caixa de texto
    y=1.18,  # posição y da caixa de texto
    showarrow=False,
)

def anotacoes_consumo(produto):
    anotacao = dict(
        text=(
            f"Capacidade Tanque: {tanque} Litros<br>"
            f"Distância por Dia: {dist_dia} Km<br>"
            f"Consumo Veículo: {km_litro[produto]} Km/l<br>"
            f"Abastecimento a cada: {round((tanque * km_litro[produto] / dist_dia), 1)} dias"
            ),
        xref="paper",
        yref="paper",
        x=0,  # posição x da caixa de texto
        y=1.18,  # posição y da caixa de texto
        showarrow=False,
    )
    return anotacao

buttons = [
    dict(
        label=produtos[0],
        method="update",
        args=[
            {"visible": [True, False, False, False, False, False]},
            {"annotations": [anotacoes_consumo(fig.data[0].name)]}
        ],
    ),
    dict(
        label=produtos[1],
        method="update",
        args=[
            {"visible": [False, True, False, False, False, False]},
            {"annotations": [anotacoes_consumo(fig.data[1].name)]}
        ],
    ),
    dict(
        label=produtos[2],
        method="update",
        args=[
            {"visible": [False, False, True, False, False, False]},
            {"annotations": [anotacoes_consumo(fig.data[2].name)]}
        ],
    ),
    dict(
        label=produtos[3],
        method="update",
        args=[
            {"visible": [False, False, False, True, False, False]},
            {"annotations": [anotacoes_consumo(fig.data[3].name)]}
        ],
    ),
    dict(
        label=produtos[4],
        method="update",
        args=[
            {"visible": [False, False, False, False, True, False]},
            {"annotations": [anotacoes_consumo(fig.data[4].name)]}
        ],
    ),
    dict(
        label=produtos[5],
        method="update",
        args=[
            {"visible": [False, False, False, False, False, True]},
            {"annotations": [anotacoes_consumo(fig.data[5].name)]}
        ],
    ),
]

# Configurações do gráfico
fig.update_layout(
    height=600,
    width=1300,
    title={
        "text": "Gasto Mensal com Abastecimento",  # Título do gráfico
        "y": 0.98,  # Posição vertical do título (0 a 1)
        "x": 0.5,  # Posição horizontal do título (0 a 1)
        "xanchor": "center",  # Ancoragem horizontal
        "yanchor": "top",  # Ancoragem vertical
        "font": {
            "family": "Arial, sans-serif",
            "size": 26,
            "color": "black",
            "weight": "bold",
        },
    },
    margin=dict(l=50, r=20, t=100, b=0),  # Margem da área de plotagem
    updatemenus=[  # Botões do Filtro. Método menos eficiente
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
            direction="left",
            x=0.5,  # Posição horizontal dos botões (0 a 1)
            y=1.1,  # Posição vertical dos botões (0 a 1)
            xanchor="center",
            yanchor="top",
            active=1,
        )
    ],
)

# Configurar eixo X
fig.update_xaxes(
    title_text="",
    tickfont=dict(family="Arial", size=12, color="black", weight="bold"),
)

# Configurar eixo Y
fig.update_yaxes(
    title_text="",
    tickfont=dict(family="Arial", size=12, color="black", weight="bold"),
    tickprefix="R$ ",
    tickformat=".0f"
)

fig.show()

## <h1 style="font-size:16px">Gasto Anual</h1>

In [ ]:
# Iniciar instância da figura
fig = go.Figure()

produto_inicial = "GASOLINA"

# Para cada produto irá gerar um gráfico da Data pelo Valor de Venda
for produto in produtos:

    filtrado = abastecimentos(df_, calendario_df, produto, tanque, km_litro, dist_dia)
    filtrado["Ano"] = filtrado["Data"].dt.year
    filtrado = filtrado.groupby("Ano").agg({"Total": "sum"}).reset_index()
    visible = produto == produto_inicial

    fig.add_trace(
        go.Bar(
            x=filtrado["Ano"].astype(str),
            y=filtrado["Total"],
            name=produto,
            visible=visible,
            text=filtrado["Total"],
            textposition="outside",
            texttemplate="R$ %{y:.2f}",
            textfont=dict(family="Arial", size=14, color="black"),
            hovertemplate=("<b>Ano:</b> %{x}<br>" "<b>Valor:</b> R$ %{y:.2f}<br>"),
        ),
    )


def anotacoes_consumo(produto):
    anotacao = dict(
        text=(
            f"Capacidade Tanque: {tanque} Litros<br>"
            f"Distância por Dia: {dist_dia} Km<br>"
            f"Consumo Veículo: {km_litro[produto]} Km/l<br>"
            f"Abastecimento a cada: {round((tanque * km_litro[produto] / dist_dia), 1)} dias"
        ),
        xref="paper",
        yref="paper",
        x=0,  # posição x da caixa de texto
        y=1.18,  # posição y da caixa de texto
        showarrow=False,
    )
    return anotacao


# Adicione uma anotação (caixa de texto)
fig.add_annotation(anotacoes_consumo(produto_inicial))

def xlim(produto):
    df1 = abastecimentos(df_, calendario_df, produto, tanque, km_litro, dist_dia)
    df1["Ano"] = df1["Data"].dt.year
    df1 = df1.groupby("Ano").agg({"Total": "sum"}).reset_index()
    xlim = df1["Total"].max().tolist()
    lim = [0, xlim * 1.1]
    return lim

buttons = [
    dict(
        label=produtos[0],
        method="update",
        args=[
            {"visible": [True, False, False, False, False, False]},
            {"annotations": [anotacoes_consumo(fig.data[0].name)]},
            {"xaxis.range": xlim(produtos[0])},
        ],
    ),
    dict(
        label=produtos[1],
        method="update",
        args=[
            {"visible": [False, True, False, False, False, False]},
            {"annotations": [anotacoes_consumo(fig.data[1].name)]},
            {"xaxis.range": xlim(produtos[1])},
        ],
    ),
    dict(
        label=produtos[2],
        method="update",
        args=[
            {"visible": [False, False, True, False, False, False]},
            {"annotations": [anotacoes_consumo(fig.data[2].name)]},
            {"xaxis.range": xlim(produtos[2])},
        ],
    ),
    dict(
        label=produtos[3],
        method="update",
        args=[
            {"visible": [False, False, False, True, False, False]},
            {"annotations": [anotacoes_consumo(fig.data[3].name)]},
            {"xaxis.range": xlim(produtos[3])},
        ],
    ),
    dict(
        label=produtos[4],
        method="update",
        args=[
            {"visible": [False, False, False, False, True, False]},
            {"annotations": [anotacoes_consumo(fig.data[4].name)]},
            {"xaxis.range": xlim(produtos[4])},
        ],
    ),
    dict(
        label=produtos[5],
        method="update",
        args=[
            {"visible": [False, False, False, False, False, True]},
            {"annotations": [anotacoes_consumo(fig.data[5].name)]},
            {"xaxis.range": xlim(produtos[5])},
        ],
    ),
]

# Configurações do gráfico
fig.update_layout(
    height=600,
    width=1100,
    title={
        "text": f"Gasto Anual com Abastecimento",  # Título do gráfico
        "y": 0.97,  # Posição vertical do título (0 a 1)
        "x": 0.5,  # Posição horizontal do título (0 a 1)
        "xanchor": "center",  # Ancoragem horizontal
        "yanchor": "top",  # Ancoragem vertical
        "font": {
            "family": "Arial, sans-serif",
            "size": 26,
            "color": "black",
            "weight": "bold",
        },
    },
    margin=dict(l=50, r=20, t=100, b=20),  # Margem da área de plotagem
    updatemenus=[  # Botões do Filtro. Método menos eficiente
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
            direction="left",
            x=0.5,  # Posição horizontal dos botões (0 a 1)
            y=1.1,  # Posição vertical dos botões (0 a 1)
            xanchor="center",
            yanchor="top",
            active=1,
        )
    ],
    xaxis=dict(
        title="",
        tickfont=dict(
            family="Arial",  # Escolha da fonte
            size=16,  # Tamanho da fonte
            color="black",  # Cor da fonte
            weight="bold",  # Aplicação de negrito (não diretamente suportado)
        ),
    ),
    yaxis=dict(
        range=xlim(produto_inicial),
        showticklabels=False,
    ),
)


fig.show()

# <h1 style="font-size:16px">Capacidade de Compra Salário Mínimo</h1>

In [ ]:
df_ = (
    df.groupby(["Ano", "Mes", "Mes Nome", "Produto"])
    .agg({"Valor de Venda": "mean"})
    .reset_index()
)

# Create a column with the buy capacity based on the minimum salary in Brazil
df_["Litros/Salario Minimo"] = df_["Valor de Venda"].apply(
    lambda x: round(salary_min[str(df_["Ano"].iloc[0])] / x, 2) if x != 0 else 0
)

df_["Mes/Ano"] = df_["Mes Nome"] + "/" + df_["Ano"].astype(str)

# Iniciar instância da figura
fig = go.Figure()

produto_inicial = "GASOLINA"


def plot_line(df, y: str, name: str, visible):
    plot = go.Scatter(
        x=df["Mes/Ano"],
        y=df[y],
        name=name,
        mode="lines",
        visible=visible,
        hovertemplate=(
            "<b>Data:</b> %{x}<br>"
            "<b>Litros:</b> %{y:.2f}<br>"
        ),
    )
    return plot


# Para cada produto irá gerar um gráfico da Data pelo Valor de Venda
for produto in produtos:
    filtrado = df_[df_["Produto"] == produto]
    visible = produto == produto_inicial
    trace1 = plot_line(filtrado, "Litros/Salario Minimo", produto, visible)

    fig.add_trace(trace1)

buttons = [
    dict(
        label=produtos[0],
        method="update",
        args=[
            {"visible": [True, False, False, False, False, False]},
        ],
    ),
    dict(
        label=produtos[1],
        method="update",
        args=[
            {"visible": [False, True, False, False, False, False]},
        ],
    ),
    dict(
        label=produtos[2],
        method="update",
        args=[
            {"visible": [False, False, True, False, False, False]},
        ],
    ),
    dict(
        label=produtos[3],
        method="update",
        args=[
            {"visible": [False, False, False, True, False, False]},
        ],
    ),
    dict(
        label=produtos[4],
        method="update",
        args=[
            {"visible": [False, False, False, False, True, False]},
        ],
    ),
    dict(
        label=produtos[5],
        method="update",
        args=[
            {"visible": [False, False, False, False, False, True]},
        ],
    ),
]

# Configurações do gráfico
fig.update_layout(
    height=600,
    width=1300,
    title={
        "text": "Litros por Salário Mínimo",  # Título do gráfico
        "y": 0.98,  # Posição vertical do título (0 a 1)
        "x": 0.5,  # Posição horizontal do título (0 a 1)
        "xanchor": "center",  # Ancoragem horizontal
        "yanchor": "top",  # Ancoragem vertical
        "font": {
            "family": "Arial, sans-serif",
            "size": 26,
            "color": "black",
            "weight": "bold",
        },
    },
    margin=dict(l=50, r=20, t=100, b=0),  # Margem da área de plotagem
    updatemenus=[  # Botões do Filtro. Método menos eficiente
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
            direction="left",
            x=0.5,  # Posição horizontal dos botões (0 a 1)
            y=1.1,  # Posição vertical dos botões (0 a 1)
            xanchor="center",
            yanchor="top",
            active=1,
        )
    ],
)

# Configurar eixo X
fig.update_xaxes(
    title_text="",
    tickfont=dict(family="Arial", size=12, color="black", weight="bold"),
)

# Configurar eixo Y
fig.update_yaxes(
    title_text="",
    tickfont=dict(family="Arial", size=14, color="black", weight="bold"),
    tickformat=".0f",
)

fig.show()

# <h1 style="font-size:16px">Função para Power Bi</h1>

In [ ]:
tanque = 50
km_litro = {
    "GASOLINA": 12,
    "ETANOL": 9,
    "DIESEL": 15,
    "DIESEL S10": 16,
    "GASOLINA ADITIVADA": 12,
    "GNV": 15,
}
dist_dia = 60


def calculo_abastecimento(df, tanque, km_litro, distancia_dia):

    produtos = df["Produto"].unique()

    def criar_calendario():

        def obter_nome_dia_semana(data):
            locale.setlocale(locale.LC_TIME, "en_US.UTF-8")
            dias_semana_pt = {
                "Monday": "segunda-feira",
                "Tuesday": "terça-feira",
                "Wednesday": "quarta-feira",
                "Thursday": "quinta-feira",
                "Friday": "sexta-feira",
                "Saturday": "sábado",
                "Sunday": "domingo",
            }

            nome_dia_em_ingles = calendar.day_name[data.weekday()]

            return dias_semana_pt[nome_dia_em_ingles]

        def criar_calendario_diario_dataframe(ano_inicio, ano_fim):
            datas = pd.date_range(start=f"{ano_inicio}-01-01", end=f"{ano_fim}-12-31")
            calendario = pd.DataFrame(datas, columns=["Data"])
            calendario["Dia da Semana"] = calendario["Data"].apply(
                obter_nome_dia_semana
            )

            return calendario

        calendario_df = criar_calendario_diario_dataframe(2022, 2024)

        calendario_df = calendario_df[
            ~calendario_df["Dia da Semana"].isin(["sábado", "domingo"])
        ]

        feriados = pd.read_excel(
            "https://www.anbima.com.br/feriados/arqs/feriados_nacionais.xls"
        )

        feriados.dropna(inplace=True)

        feriados["Data"] = pd.to_datetime(feriados["Data"], format="%Y/%m/%d")

        feriados = feriados[
            (feriados["Data"].dt.year.isin(calendario_df["Data"].dt.year.unique()))
        ]

        for ano in calendario_df["Data"].dt.year.unique():
            aniv_sp = pd.to_datetime(f"{ano}-01-25")
            feriados.loc[len(feriados)] = [
                aniv_sp,
                obter_nome_dia_semana(aniv_sp).lower(),
                "Aniversário de São Paulo",
            ]

        feriados = feriados.sort_values(by="Data").reset_index(drop=True)

        feriados = feriados[~feriados["Dia da Semana"].isin(["sábado", "domingo"])]

        calendario_df = (
            calendario_df.merge(feriados, on="Data", how="left")
            .drop(columns="Dia da Semana_y")
            .rename(columns={"Dia da Semana_x": "Dia da Semana"})
        )

        calendario_df["Feriado"] = calendario_df["Feriado"].fillna("Dia Útil")

        calendario_df = calendario_df[calendario_df["Feriado"] == "Dia Útil"]

        calendario_df.drop(columns="Feriado", inplace=True)

        return calendario_df

    df_calendario = criar_calendario()

    def abastecimentos(produto):

        df_produto = df[df["Produto"] == produto]
        df_abast = df_calendario.merge(df_produto, on="Data", how="left").bfill()
        df_abast.loc[0, "Data"] = df_abast.loc[0, "Data"] + timedelta(hours=9)
        df_abast.set_index("Data", inplace=True)

        def intervalo_dias(produto, tanque, km_litro, distancia_dia):
            intervalo = round((tanque * km_litro[produto] / distancia_dia), 1)
            return pd.Timedelta(days=intervalo)

        intervalo = intervalo_dias(produto, tanque, km_litro, distancia_dia)

        def ajustar_data_hora(data):
            hora_inicio = pd.Timestamp("07:00").time()
            hora_fim = pd.Timestamp("18:00").time()

            if isinstance(data, pd.Timestamp):
                hora = data.time()
            else:
                raise TypeError("O objeto 'data' deve ser do tipo pd.Timestamp.")

            if hora < hora_inicio:
                data = data.replace(hour=7, minute=0, second=0, microsecond=0)
            elif hora > hora_fim:
                diferenca = data - data.replace(
                    hour=18, minute=0, second=0, microsecond=0
                )
                data = data + pd.Timedelta(days=1)
                data = data.replace(hour=7, minute=0, second=0, microsecond=0)
                data += diferenca

                while data.weekday() >= 5:
                    data += pd.Timedelta(days=1)
            return data

        datas_selecionadas = []
        data_atual = df_abast.index.min()

        while data_atual <= df_abast.index.max():
            data_atual = ajustar_data_hora(data_atual)
            datas_selecionadas.append(data_atual)
            data_atual += intervalo

        abastecimento = df_abast.reindex(datas_selecionadas, method="nearest")

        abastecimento["Total"] = abastecimento["Valor de Venda"].mul(tanque)

        abastecimento["Valor de Venda"] = abastecimento["Valor de Venda"].replace(
            ".", "."
        )
        abastecimento["Total"] = abastecimento["Total"].replace(".", ",")

        abastecimento.reset_index(inplace=True)

        return abastecimento

    def gerar_dataframe():
        df_final = abastecimentos(produtos[0])

        for produto in produtos[1:]:
            df_2 = abastecimentos(produto)
            df_final = pd.concat([df_final, df_2], axis=0)

        return df_final

    df_final = gerar_dataframe()

    return df_final